# Testing for E1 - COGNITION 7Subjects
fistly define the UUID of the experiment to use

calculating with classical F1 to compare with our F1* in the reviewers' response

In [1]:
experimentId="09aa67d8-c865-4a95-a7c5-de7b6adadbce"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## Overrride metrics to do classical F1 instead of F1*

In [3]:
def annotationPairToMetrics(annotations,detections,thresIoU=0.3):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1 the old way
    N=len(annotations)
    M=len(detections)
    TP=np.sum(np.max(binarized,axis=0))
    FN=N-TP
    FP=M-TP

    outF1=(2*TP)/(2*TP+FN+FP)
    recall=TP/N
    precision=TP/M

    #calculateF1*
    """ outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections) """
    return outF1,recall,precision

## load cognition

In [4]:
samplerate=200  #Should rethink this

In [5]:
annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath)

In [6]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [7]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Load experiment results

In [8]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [9]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,9e8c92c9-d012-4e46-a60f-74540df5103d,0.008075
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,16258fa7-20b7-495e-a32f-75aec610ef66,0.005000
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,3654a0cd-77b5-4bcf-8238-54296d7c1a27,0.012035
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,8cec6075-9679-4f7b-b5c7-d6ff36147e79,0.010340
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,5401bdbe-22c1-42a0-9740-d2921a271387,0.010340
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,2d237bf5-9265-439a-827b-f1411cb4fc3a,0.006673
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,8bf430c8-bbc9-4ee2-927e-75f7a5e76570,0.006822


In [10]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                                 \
                        count      mean       std    min       25%       50%   
criteriumName                                                                  
E1                        7.0  0.008469  0.002511  0.005  0.006748  0.008075   

                                  
                   75%       max  
criteriumName                     
E1             0.01034  0.012035

In [11]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [12]:
hyperClose=0.1
hyperDuration=0.3
hyperThres=0.4
hyperDepth=10

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [13]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,9e8c92c9-d012-4e46-a60f-74540df5103d,0.008075
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,16258fa7-20b7-495e-a32f-75aec610ef66,0.005000
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,3654a0cd-77b5-4bcf-8238-54296d7c1a27,0.012035
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,8cec6075-9679-4f7b-b5c7-d6ff36147e79,0.010340
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,5401bdbe-22c1-42a0-9740-d2921a271387,0.010340
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,2d237bf5-9265-439a-827b-f1411cb4fc3a,0.006673
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,8bf430c8-bbc9-4ee2-927e-75f7a5e76570,0.006822


In [14]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<5]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 7
*************************
2 of 7
*************************
3 of 7
*************************
4 of 7
*************************
5 of 7
*************************
6 of 7
*************************
7 of 7


In [15]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [16]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [17]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [18]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [19]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [20]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [21]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0002     1.0  0.622691 NaN  0.622691  0.622691  0.622691            1.0   
0003     1.0  0.851014 NaN  0.851014  0.851014  0.851014            1.0   
0004     1.0  0.775229 NaN  0.775229  0.775229  0.775229            1.0   
0005     1.0  0.618297 NaN  0.618297  0.618297  0.618297            1.0   
0006     1.0  0.690519 NaN  0.690519  0.690519  0.690519            1.0   
0007     1.0  0.613050 NaN  0.613050  0.613050  0.613050            1.0   
0008     1.0  0.513514 NaN  0.513514  0.513514  0.513514            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0002  0.848921 NaN  0.848921  0.848921  0.848921         1.0  0.491667 NaN   
0003  0.837900 NaN  0.837900  0.837900  0.837900         1.0  0.864547 NaN   
0004  0.746137 NaN  0.746137  0.746137  0.746137         1.0  0.806683 NaN   
0005  0.462992 NaN  0.462992  0.462992  0.462992         1.0  0.930380 NaN   
0006  0.579580 NaN  0.579580  0.579580  0.579580         1.0  0.853982 NaN   
0007  0.703833 NaN  0.703833  0.703833  0.703833         1.0  0.543011 NaN   
0008  0.703704 NaN  0.703704  0.703704  0.703704         1.0  0.404255 NaN   

                                    
           min       50%       max  
test                                
0002  0.491667  0.491667  0.491667  
0003  0.864547  0.864547  0.864547  
0004  0.806683  0.806683  0.806683  
0005  0.930380  0.930380  0.930380  
0006  0.853982  0.853982  0.853982  
0007  0.543011  0.543011  0.543011  
0008  0.404255  0.404255  0.404255

by-event

In [22]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0002,0.622691,0.848921,0.491667
1,0003,0.851014,0.837900,0.864547
2,0004,0.775229,0.746137,0.806683
3,0005,0.618297,0.462992,0.930380
4,0006,0.690519,0.579580,0.853982
5,0007,0.613050,0.703833,0.543011
6,0008,0.513514,0.703704,0.404255


In [23]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           6.691877e-01
eventPrecision    6.975808e-01
eventRecall       6.992177e-01
dtype: float64

In [24]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_18920\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.113210
eventPrecision    0.137831
eventRecall       0.212427
dtype: float64

by-sample

In [25]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.138860e-01
precision    6.750247e-01
recall       6.162432e-01
dtype: float64

In [26]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_18920\634953098.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.105946
precision    0.133751
recall       0.200678
dtype: float64

In [27]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [28]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### Restricted to N2

In [29]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)
    testSubjectId=row.test
   
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    ##################################################
    hypnogram=loadCOGNITIONHypnogram(testSubjectId,cognipath)
    testLabels=testLabels*(hypnogram==2)
    ##################################################

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    ##################################################
    rawLabels=rawLabels*(hypnogram==2)
    ##################################################

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<5]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 7
*************************
2 of 7
*************************
3 of 7
*************************
4 of 7
*************************
5 of 7
*************************
6 of 7
*************************
7 of 7


In [30]:
dumpPickle('experimentModelsTestN2_E1_temp.pkl',experimentModels)

In [31]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [32]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [33]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [34]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [35]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [36]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0002     1.0  0.660256 NaN  0.660256  0.660256  0.660256            1.0   
0003     1.0  0.857837 NaN  0.857837  0.857837  0.857837            1.0   
0004     1.0  0.788698 NaN  0.788698  0.788698  0.788698            1.0   
0005     1.0  0.618378 NaN  0.618378  0.618378  0.618378            1.0   
0006     1.0  0.693487 NaN  0.693487  0.693487  0.693487            1.0   
0007     1.0  0.628752 NaN  0.628752  0.628752  0.628752            1.0   
0008     1.0  0.535211 NaN  0.535211  0.535211  0.535211            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0002  0.865546 NaN  0.865546  0.865546  0.865546         1.0  0.533679 NaN   
0003  0.846523 NaN  0.846523  0.846523  0.846523         1.0  0.869458 NaN   
0004  0.755294 NaN  0.755294  0.755294  0.755294         1.0  0.825193 NaN   
0005  0.462036 NaN  0.462036  0.462036  0.462036         1.0  0.934641 NaN   
0006  0.583871 NaN  0.583871  0.583871  0.583871         1.0  0.853774 NaN   
0007  0.731618 NaN  0.731618  0.731618  0.731618         1.0  0.551247 NaN   
0008  0.703704 NaN  0.703704  0.703704  0.703704         1.0  0.431818 NaN   

                                    
           min       50%       max  
test                                
0002  0.533679  0.533679  0.533679  
0003  0.869458  0.869458  0.869458  
0004  0.825193  0.825193  0.825193  
0005  0.934641  0.934641  0.934641  
0006  0.853774  0.853774  0.853774  
0007  0.551247  0.551247  0.551247  
0008  0.431818  0.431818  0.431818

by-event

In [37]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0002,0.660256,0.865546,0.533679
1,0003,0.857837,0.846523,0.869458
2,0004,0.788698,0.755294,0.825193
3,0005,0.618378,0.462036,0.934641
4,0006,0.693487,0.583871,0.853774
5,0007,0.628752,0.731618,0.551247
6,0008,0.535211,0.703704,0.431818


In [38]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           6.832314e-01
eventPrecision    7.069416e-01
eventRecall       7.142584e-01
dtype: float64

In [39]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_18920\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.109032
eventPrecision    0.143016
eventRecall       0.201404
dtype: float64

by-sample

In [40]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.253867e-01
precision    6.813343e-01
recall       6.291909e-01
dtype: float64

In [41]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_18920\634953098.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.100984
precision    0.136692
recall       0.192839
dtype: float64

In [42]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [43]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()